- https://api.python.langchain.com/en/latest/community/chat_models/langchain_community.chat_models.llamacpp.ChatLlamaCpp.html

In [1]:
import multiprocessing
from langchain_community.chat_models import ChatLlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# RAG 체인 구성
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

from dotenv import dotenv_values

In [2]:
config = dotenv_values(
    dotenv_path = "../.env"
)

In [3]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# RAG 시스템을 위한 LLM 설정 (KV 캐시 최적화 포함)
llm = ChatLlamaCpp(
    temperature=0.1,  # RAG에서는 더 일관된 답변을 위해 낮은 temperature 사용
    model_path=config["EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf"],
    n_ctx=16384,  # 컨텍스트 크기를 적절히 조정 (32768은 메모리 부담이 큼)
    n_gpu_layers=8,
    n_batch=32,  # KV 캐시 최적화를 위해 더 작은 배치 크기
    max_tokens=512,  # 더 긴 답변을 위해 토큰 수 증가
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.1,  # RAG에서는 적당한 반복 패널티
    top_p=0.9,  # RAG에서는 더 높은 top_p로 다양한 답변 생성
    verbose=False,  # RAG에서는 verbose 비활성화로 출력 정리
    callback_manager=callback_manager,
    
    # KV 캐시 최적화를 위한 설정들
    use_mlock=True,  # 메모리 잠금으로 성능 향상
    use_mmap=True,   # 메모리 맵핑 사용
    
    # 추가적인 KV 캐시 관련 설정들
    # n_keep=-1,     # 모든 토큰을 캐시에 유지 (선택사항)
    # rope_scaling_type=1,  # RoPE 스케일링 (긴 컨텍스트용)
    # rope_freq_base=10000,  # RoPE 주파수 기본값
)

In [5]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 샘플 문서 데이터 (실제로는 파일에서 로드)
documents = [
    "인공지능(AI)은 컴퓨터 시스템이 인간의 지능을 모방하여 학습하고 추론할 수 있도록 하는 기술입니다.",
    "머신러닝은 AI의 한 분야로, 데이터로부터 패턴을 학습하여 예측이나 분류를 수행합니다.",
    "딥러닝은 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 하위 분야입니다.",
    "자연어처리(NLP)는 인간의 언어를 컴퓨터가 이해하고 처리할 수 있도록 하는 AI 기술입니다.",
    "컴퓨터 비전은 이미지나 비디오에서 의미 있는 정보를 추출하는 AI 분야입니다."
]

# 문서 분할
texts = text_splitter.create_documents(documents)

In [6]:
# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name=config["multilingual-e5-small-ko"],
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [7]:
# 벡터 스토어 생성
vectorstore = FAISS.from_documents(texts, embeddings)

In [8]:
# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False
)

In [9]:
# 프롬프트 구성
question = "머신러닝에 대해 알려줘"
prompt = f"<|im_start|>system\n당신은 도움이 되는 AI 어시스턴트입니다. 질문에 대해 3문장 정도 정확하고 유용한 답변을 제공해주세요.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"


In [10]:
response = qa_chain.invoke(prompt)

머신러닝은 컴퓨터 시스템이 데이터로부터 학습하여 패턴을 인식하고 예측 또는 결정을 내리는 기술입니다. 이 과정에서 알고리즘은 반복적인 훈련을 통해 성능을 향상시킵니다. 주요 응용 분야로는 추천 시스템, 이미지 및 음성 인식, 자연어 처리 등이 있습니다.